In [ ]:
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
import re
import csv
from pathlib import Path
import traceback
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Try to import pylsl, but make it optional
try:
    import pylsl
    LSL_AVAILABLE = True
except ImportError:
    LSL_AVAILABLE = False
    print("Warning: pylsl module not found. LSL functionality will be disabled.")

# ======= CONFIGURATION VARIABLES =======
# Directory paths
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
AUDIO_OUTPUT_DIR = os.path.join(BASE_DIR, "PPS_Experiment_Module", "ExperimentAudio")
DESIGN_OUTPUT_DIR = os.path.join(BASE_DIR, "PPS_Experiment_Module", "ExperimentLog")
DATA_LOG_DIR = os.path.join(BASE_DIR, "PPS_Experiment_Module", "DataLogs")
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")

# Response window in seconds
RESPONSE_WINDOW = 1.5

# LSL Stream Configuration (if available)
LSL_STREAM_NAME_CLICKS = "MouseClicks"
LSL_STREAM_NAME_MARKERS = "ExperimentMarkers"
LSL_STREAM_TYPE = "Markers"
LSL_SAMPLING_RATE = 0  # Irregular sampling rate

# Debug mode - set to True to print more detailed information
DEBUG_MODE = True

# Ensure directories exist
os.makedirs(AUDIO_OUTPUT_DIR, exist_ok=True)
os.makedirs(DESIGN_OUTPUT_DIR, exist_ok=True)
os.makedirs(DATA_LOG_DIR, exist_ok=True)
os.makedirs(PRACTICE_STIMULI_DIR, exist_ok=True)

# Set up logging
log_filename = os.path.join(DATA_LOG_DIR, f"experiment_log_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
if not os.path.exists(os.path.dirname(log_filename)):
    os.makedirs(os.path.dirname(log_filename), exist_ok=True)

import logging
logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s',
                   handlers=[
                       logging.FileHandler(log_filename),
                       logging.StreamHandler()
                   ])

class PPSExperimentRunner:
    def __init__(self):
        # Initialize variables
        self.participant_id = None
        self.participant_initials = None
        self.experiment_start_time = None
        self.stop_flag = False
        self.pause_flag = False
        self.practice_passed = False
        
        # Audio files and data
        self.looming_audio_file = None
        self.tactile_audio_file = None
        self.design_data = None
        
        # Audio streams
        self.looming_stream = None
        self.tactile_stream = None
        
        # Response tracking
        self.responses = []
        self.missed_trials = []
        self.trial_timeline = {}  # Maps timestamps to trial information
        self.timeline_markers = []  # Visualization markers
        self.click_count = 0
        
        # For timeline visualization
        self.visual_canvas = None
        self.progress_line = None
        self.timeline_end_x = 0
        self.timeline_y = 0
        
        # LSL outlets
        self.marker_outlet = None
        self.click_outlet = None
        self.lsl_enabled = LSL_AVAILABLE
        
        # File paths
        self.response_csv_file = None
        
        # Root window
        self.root = None
        
        logging.info("PPS Experiment Runner initialized")
    
    def get_participant_info(self):
        """Get participant ID and name initials."""
        # Create root window
        root = tk.Tk()
        root.title("Participant Information")
        root.geometry("500x350")
        
        # Center the window
        root.update_idletasks()
        width = root.winfo_width()
        height = root.winfo_height()
        x = (root.winfo_screenwidth() // 2) - (width // 2)
        y = (root.winfo_screenheight() // 2) - (height // 2)
        root.geometry(f'+{x}+{y}')
        
        # Create main frame
        main_frame = ttk.Frame(root, padding="20")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Header
        ttk.Label(main_frame, text="Enter Participant Information", font=("Arial", 14, "bold")).pack(pady=10)
        
        # Participant ID
        id_frame = ttk.Frame(main_frame)
        id_frame.pack(fill=tk.X, pady=10)
        
        ttk.Label(id_frame, text="Participant ID (0-99):").pack(side=tk.LEFT, padx=5)
        id_var = tk.StringVar()
        id_entry = ttk.Entry(id_frame, textvariable=id_var, width=5)
        id_entry.pack(side=tk.LEFT, padx=5)
        
        # First name initial
        first_frame = ttk.Frame(main_frame)
        first_frame.pack(fill=tk.X, pady=10)
        
        ttk.Label(first_frame, text="First letter of first name:").pack(side=tk.LEFT, padx=5)
        first_var = tk.StringVar()
        first_entry = ttk.Entry(first_frame, textvariable=first_var, width=3)
        first_entry.pack(side=tk.LEFT, padx=5)
        
        # Last name initial
        last_frame = ttk.Frame(main_frame)
        last_frame.pack(fill=tk.X, pady=10)
        
        ttk.Label(last_frame, text="Last letter of last name:").pack(side=tk.LEFT, padx=5)
        last_var = tk.StringVar()
        last_entry = ttk.Entry(last_frame, textvariable=last_var, width=3)
        last_entry.pack(side=tk.LEFT, padx=5)
        
        # Preview
        preview_frame = ttk.LabelFrame(main_frame, text="Participant Identifier")
        preview_frame.pack(fill=tk.X, pady=15)
        
        preview_var = tk.StringVar(value="XX00")
        preview_label = ttk.Label(preview_frame, textvariable=preview_var, font=("Arial", 16, "bold"))
        preview_label.pack(pady=10)
        
        # Update preview when entries change
        def update_preview(*args):
            try:
                first = first_var.get().upper()
                last = last_var.get().upper()
                pid = id_var.get().zfill(2)
                
                if first and last and pid:
                    preview_var.set(f"{first[0]}{last[0]}{pid}")
            except:
                pass
        
        id_var.trace_add("write", update_preview)
        first_var.trace_add("write", update_preview)
        last_var.trace_add("write", update_preview)
        
        # Buttons
        button_frame = ttk.Frame(main_frame)
        button_frame.pack(pady=20)
        
        # Function to confirm entry
        result = [None, None]  # To store results
        
        def on_confirm():
            # Validate input
            pid = id_var.get().strip()
            first = first_var.get().strip().upper()
            last = last_var.get().strip().upper()
            
            if not pid or not pid.isdigit() or int(pid) < 0 or int(pid) > 99:
                messagebox.showerror("Error", "Participant ID must be a number between 0 and 99")
                return
            
            if not first or len(first) < 1:
                messagebox.showerror("Error", "Please enter the first letter of participant's first name")
                return
            
            if not last or len(last) < 1:
                messagebox.showerror("Error", "Please enter the last letter of participant's last name")
                return
            
            # Store results and close window
            result[0] = pid
            result[1] = f"{first[0]}{last[0]}{pid.zfill(2)}"
            root.destroy()
        
        def on_cancel():
            root.destroy()
        
        ttk.Button(button_frame, text="Confirm", command=on_confirm).pack(side=tk.LEFT, padx=10)
        ttk.Button(button_frame, text="Cancel", command=on_cancel).pack(side=tk.LEFT, padx=10)
        
        # Start the GUI
        root.mainloop()
        
        # Set participant info
        if result[0] is not None:
            self.participant_id = result[0]
            self.participant_initials = result[1]
            logging.info(f"Set participant ID: {self.participant_id}, Identifier: {self.participant_initials}")
            return True
        
        return False
    
    def confirm_lsl_recorder(self):
        """Confirm that the LSL recorder is running (if LSL is available)."""
        if not self.lsl_enabled:
            logging.info("LSL not available - skipping recorder confirmation")
            return True
            
        root = tk.Tk()
        root.title("LSL Recorder Confirmation")
        root.geometry("500x300")
        
        # Center the window
        root.update_idletasks()
        width = root.winfo_width()
        height = root.winfo_height()
        x = (root.winfo_screenwidth() // 2) - (width // 2)
        y = (root.winfo_screenheight() // 2) - (height // 2)
        root.geometry(f'+{x}+{y}')
        
        # Create main frame
        main_frame = ttk.Frame(root, padding="20")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Header
        ttk.Label(main_frame, text="Important!", font=("Arial", 16, "bold")).pack(pady=5)
        
        # Warning message
        msg = ("Please confirm that the LSL recorder is running!\n\n"
              "This is CRITICAL - without the LSL recorder, no data will be saved.\n\n"
              "Have you started the LSL recorder?")
        
        ttk.Label(main_frame, text=msg, wraplength=400, justify="center").pack(pady=20)
        
        # Checkbox
        lsl_confirmed = tk.BooleanVar(value=False)
        ttk.Checkbutton(main_frame, text="Yes, I have started the LSL recorder", 
                       variable=lsl_confirmed).pack(pady=10)
        
        # Function to confirm
        def on_confirm():
            if not lsl_confirmed.get():
                messagebox.showerror("Error", "You must confirm the LSL recorder is running.")
                return
            
            self.lsl_recorder_confirmed = True
            root.destroy()
        
        def on_cancel():
            root.destroy()
        
        # Buttons
        button_frame = ttk.Frame(main_frame)
        button_frame.pack(pady=20)
        
        ttk.Button(button_frame, text="Confirm", command=on_confirm).pack(side=tk.LEFT, padx=10)
        ttk.Button(button_frame, text="Cancel", command=on_cancel).pack(side=tk.LEFT, padx=10)
        
        # Start the GUI
        root.mainloop()
        
        return hasattr(self, 'lsl_recorder_confirmed') and self.lsl_recorder_confirmed
    
    def find_participant_files(self):
        """Find the participant's audio and design files based on the participant ID."""
        # Look for design file
        design_file_pattern = f"participant_{self.participant_id}_design.csv"
        design_file_path = os.path.join(DESIGN_OUTPUT_DIR, design_file_pattern)
        
        if not os.path.exists(design_file_path):
            # Try to find files that match the pattern
            design_files = [f for f in os.listdir(DESIGN_OUTPUT_DIR) 
                          if f.startswith(f"participant_{self.participant_id}_") and f.endswith(".csv")]
            
            if not design_files:
                raise FileNotFoundError(f"No design file found for participant {self.participant_id}")
            
            design_file_path = os.path.join(DESIGN_OUTPUT_DIR, design_files[0])
        
        self.design_file_path = design_file_path
        logging.info(f"Found design file: {self.design_file_path}")
        
        # Load design data
        self.design_data = pd.read_csv(self.design_file_path)
        logging.info(f"Loaded design data with {len(self.design_data)} trials")
        
        # Look for looming audio file
        looming_file_pattern = f"participant_{self.participant_id}_design_looming.wav"
        looming_file_path = os.path.join(AUDIO_OUTPUT_DIR, looming_file_pattern)
        
        if not os.path.exists(looming_file_path):
            # Try to find files that match by participant ID
            looming_files = [f for f in os.listdir(AUDIO_OUTPUT_DIR) 
                           if f.startswith(f"participant_{self.participant_id}_") and "looming" in f.lower() and f.endswith(".wav")]
            
            if not looming_files:
                raise FileNotFoundError(f"No looming audio file found for participant {self.participant_id}")
            
            looming_file_path = os.path.join(AUDIO_OUTPUT_DIR, looming_files[0])
        
        self.looming_audio_file = looming_file_path
        logging.info(f"Found looming audio file: {self.looming_audio_file}")
        
        # Look for tactile audio file
        tactile_file_pattern = f"participant_{self.participant_id}_design_tactile.wav"
        tactile_file_path = os.path.join(AUDIO_OUTPUT_DIR, tactile_file_pattern)
        
        if not os.path.exists(tactile_file_path):
            # Try to find files that match by participant ID
            tactile_files = [f for f in os.listdir(AUDIO_OUTPUT_DIR) 
                           if f.startswith(f"participant_{self.participant_id}_") and "tactile" in f.lower() and f.endswith(".wav")]
            
            if not tactile_files:
                raise FileNotFoundError(f"No tactile audio file found for participant {self.participant_id}")
            
            tactile_file_path = os.path.join(AUDIO_OUTPUT_DIR, tactile_files[0])
        
        self.tactile_audio_file = tactile_file_path
        logging.info(f"Found tactile audio file: {self.tactile_audio_file}")
        
        # Build trial timeline from design data
        self.build_trial_timeline()
        
        return True
    
    def build_trial_timeline(self):
        """Build a timeline of trials from the design data."""
        self.trial_timeline = {}
        
        if self.design_data is not None:
            for _, row in self.design_data.iterrows():
                # Convert timestamp to seconds
                if 'timestamp_after_jitter' in row:
                    time_str = row['timestamp_after_jitter']
                    match = re.match(r'(\d+):(\d+\.\d+)', time_str)
                    if match:
                        minutes, seconds = match.groups()
                        time_sec = int(minutes) * 60 + float(seconds)
                        
                        # Add to timeline
                        self.trial_timeline[time_sec] = {
                            'trial_number': row['trial_number'],
                            'trial_type': row['trial_type'],
                            'stimulus_type': row['stimulus_type'],
                            'soa_value_ms': row['soa_value_ms'] if 'soa_value_ms' in row else 0
                        }
                
                # Also add tactile timeline for trials with tactile stimuli
                if 'timestamp_with_soa' in row and row['trial_type'] != 'catch':
                    time_str = row['timestamp_with_soa']
                    match = re.match(r'(\d+):(\d+\.\d+)', time_str)
                    if match:
                        minutes, seconds = match.groups()
                        tactile_time_sec = int(minutes) * 60 + float(seconds)
                        
                        # Add to timeline with additional marker for tactile
                        self.trial_timeline[tactile_time_sec] = {
                            'trial_number': row['trial_number'],
                            'trial_type': row['trial_type'],
                            'stimulus_type': row['stimulus_type'],
                            'soa_value_ms': row['soa_value_ms'] if 'soa_value_ms' in row else 0,
                            'is_tactile': True
                        }
        
        logging.info(f"Built trial timeline with {len(self.trial_timeline)} entries")
        return self.trial_timeline
    
    def setup_lsl_streams(self):
        """Set up the Lab Streaming Layer (LSL) outlets for markers and mouse clicks."""
        if not self.lsl_enabled:
            logging.info("LSL not available - skipping stream setup")
            return True
            
        try:
            # Create LSL stream for experiment markers
            marker_info = pylsl.StreamInfo(
                name=f"{LSL_STREAM_NAME_MARKERS}_{self.participant_initials}",
                type=LSL_STREAM_TYPE,
                channel_count=3,  # [timestamp, marker_type, participant_id]
                nominal_srate=LSL_SAMPLING_RATE,
                channel_format=pylsl.cf_string,
                source_id=f"markers_{self.participant_initials}"
            )
            
            # Add metadata
            marker_desc = marker_info.desc()
            marker_desc.append_child_value("participant_id", self.participant_id)
            marker_desc.append_child_value("participant_initials", self.participant_initials)
            channels = marker_desc.append_child("channels")
            channels.append_child("channel").append_child_value("label", "timestamp")
            channels.append_child("channel").append_child_value("label", "marker_type")
            channels.append_child("channel").append_child_value("label", "participant_id")
            
            # Create outlet
            self.marker_outlet = pylsl.StreamOutlet(marker_info)
            logging.info(f"Created LSL outlet for experiment markers: {LSL_STREAM_NAME_MARKERS}_{self.participant_initials}")
            
            # Create LSL stream for mouse clicks
            click_info = pylsl.StreamInfo(
                name=f"{LSL_STREAM_NAME_CLICKS}_{self.participant_initials}",
                type=LSL_STREAM_TYPE,
                channel_count=4,  # [timestamp, click_type, trial_info, participant_id]
                nominal_srate=LSL_SAMPLING_RATE,
                channel_format=pylsl.cf_string,
                source_id=f"clicks_{self.participant_initials}"
            )
            
            # Add metadata
            click_desc = click_info.desc()
            click_desc.append_child_value("participant_id", self.participant_id)
            click_desc.append_child_value("participant_initials", self.participant_initials)
            channels = click_desc.append_child("channels")
            channels.append_child("channel").append_child_value("label", "timestamp")
            channels.append_child("channel").append_child_value("label", "click_type")
            channels.append_child("channel").append_child_value("label", "trial_info")
            channels.append_child("channel").append_child_value("label", "participant_id")
            
            # Create outlet
            self.click_outlet = pylsl.StreamOutlet(click_info)
            logging.info(f"Created LSL outlet for mouse clicks: {LSL_STREAM_NAME_CLICKS}_{self.participant_initials}")
            
            return True
            
        except Exception as e:
            logging.error(f"Error setting up LSL streams: {e}")
            return False
    
    def setup_click_listener(self, window):
        """Set up a listener for mouse clicks to send to LSL and local CSV."""
        try:
            # Function to handle mouse clicks
            def on_mouse_click(event):
                # Print for debugging
                if DEBUG_MODE:
                    print(f"Mouse click detected at screen position ({event.x}, {event.y})")
                
                # Show visual feedback in the timeline view
                self.flash_click_indicator()
                
                # Ignore if we haven't started yet
                if self.experiment_start_time is None:
                    if DEBUG_MODE:
                        print("Click ignored - experiment not started")
                    return
                
                # Ignore if experiment is paused
                if self.pause_flag:
                    if DEBUG_MODE:
                        print("Click ignored - experiment paused")
                    return
                
                # Get current time since experiment start
                current_time = time.perf_counter() - self.experiment_start_time
                
                # Determine click type
                if event.num == 1:
                    click_type = "left_click"
                elif event.num == 2:
                    click_type = "middle_click"
                elif event.num == 3:
                    click_type = "right_click"
                else:
                    click_type = f"button_{event.num}"
                
                # Increment click counter
                self.click_count += 1
                try:
                    if hasattr(self, 'click_counter_text') and self.visual_canvas.winfo_exists():
                        self.visual_canvas.itemconfig(
                            self.click_counter_text, 
                            text=f"Total Clicks: {self.click_count}"
                        )
                except Exception as e:
                    if DEBUG_MODE:
                        print(f"Error updating click counter: {e}")
                
                # Add click marker to timeline
                self.add_timeline_marker(current_time, "red")
                
                # Find current trial
                current_trial = self.find_current_trial(current_time)
                
                # Record the response
                response_data = {
                    'time': current_time,
                    'timestamp': datetime.datetime.now().isoformat(),
                    'click_type': click_type,
                    'trial_number': current_trial.get('trial_number', 'unknown') if current_trial else 'unknown',
                    'trial_type': current_trial.get('trial_type', 'unknown') if current_trial else 'unknown',
                    'stimulus_type': current_trial.get('stimulus_type', 'unknown') if current_trial else 'unknown',
                    'soa_ms': current_trial.get('soa_value_ms', 0) if current_trial else 0,
                    'x': event.x,
                    'y': event.y
                }
                
                self.responses.append(response_data)
                
                # Process the response to check if it's valid
                if current_trial and 'is_tactile' in current_trial:
                    # This is a tactile stimulus point
                    last_tactile_time = self.find_last_tactile_time(current_time)
                    if last_tactile_time is not None:
                        response_latency = current_time - last_tactile_time
                        response_data['latency'] = response_latency
                        
                        # Check if response is within window
                        is_valid = response_latency <= RESPONSE_WINDOW
                        
                        if DEBUG_MODE:
                            validity = "VALID" if is_valid else "TOO SLOW"
                            print(f"Response at {current_time:.3f}s for tactile at {last_tactile_time:.3f}s - Latency: {response_latency:.3f}s ({validity})")
                
                # Write to CSV if it exists
                if hasattr(self, 'response_csv_file') and self.response_csv_file:
                    self.write_response_to_csv(response_data)
                
                # Send to LSL
                if self.lsl_enabled and self.click_outlet:
                    trial_info = f"trial_{response_data['trial_number']}" if current_trial else "unknown_trial"
                    self.click_outlet.push_sample([
                        str(current_time),
                        click_type,
                        trial_info,
                        self.participant_initials
                    ])
                    
                    if DEBUG_MODE:
                        print(f"Sent to LSL: {click_type} at {current_time:.3f}s (Trial: {trial_info})")
            
            # Bind to window
            window.bind("<Button-1>", on_mouse_click)  # Left click
            window.bind("<Button-2>", on_mouse_click)  # Middle click
            window.bind("<Button-3>", on_mouse_click)  # Right click
            
            logging.info("Mouse click listener set up")
            return True
            
        except Exception as e:
            logging.error(f"Error setting up mouse click listener: {e}")
            return False
    
    def find_current_trial(self, current_time):
        """
        Find the current trial based on the experiment time.
        
        Args:
            current_time: Time since experiment start (seconds)
            
        Returns:
            Dictionary with trial information or None
        """
        # Find the most recent trial that started before current_time
        current_trial = None
        current_trial_time = 0
        
        for time_sec, trial_info in sorted(self.trial_timeline.items()):
            if time_sec <= current_time and time_sec > current_trial_time:
                current_trial_time = time_sec
                current_trial = trial_info
        
        return current_trial
    
    def find_last_tactile_time(self, current_time):
        """
        Find the time of the last tactile stimulus before current_time.
        
        Args:
            current_time: Time since experiment start (seconds)
            
        Returns:
            Time of last tactile stimulus or None
        """
        # Find the most recent tactile stimulus
        last_tactile_time = None
        
        for time_sec, trial_info in sorted(self.trial_timeline.items()):
            if time_sec <= current_time and trial_info.get('is_tactile', False):
                last_tactile_time = time_sec
        
        return last_tactile_time
    
    def write_response_to_csv(self, response_data):
        """
        Write a response to the CSV file.
        
        Args:
            response_data: Dictionary with response information
        """
        try:
            # Check if file exists
            file_exists = os.path.exists(self.response_csv_file)
            
            with open(self.response_csv_file, 'a', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=response_data.keys())
                
                # Write header if new file
                if not file_exists:
                    writer.writeheader()
                
                writer.writerow(response_data)
                
            if DEBUG_MODE:
                print(f"Response saved to CSV: {self.response_csv_file}")
                
        except Exception as e:
            logging.error(f"Error writing response to CSV: {e}")
    
    def create_response_csv(self):
        """Create a new response CSV file with proper naming convention."""
        try:
            # Create a timestamp-based filename
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{self.participant_initials}_responses_{timestamp}.csv"
            self.response_csv_file = os.path.join(DATA_LOG_DIR, filename)
            
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(self.response_csv_file), exist_ok=True)
            
            # Initialize with header only (rows added as clicks happen)
            with open(self.response_csv_file, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    'time', 'timestamp', 'click_type', 'trial_number', 
                    'trial_type', 'stimulus_type', 'soa_ms', 'x', 'y', 'latency'
                ])
            
            logging.info(f"Created response CSV file: {self.response_csv_file}")
            return True
            
        except Exception as e:
            logging.error(f"Error creating response CSV file: {e}")
            return False
    
    def update_progress_line(self, elapsed_time):
        """Update the position of the progress line based on elapsed time."""
        try:
            # Calculate position
            timeline_start_x = 400
            timeline_width = self.timeline_end_x - timeline_start_x
            
            # Calculate max duration based on last trial time
            max_duration = 90  # Default 90 seconds
            if self.trial_timeline:
                max_duration = max(self.trial_timeline.keys()) + 30  # Add 30s buffer
            
            x_pos = timeline_start_x + min(elapsed_time / max_duration, 1.0) * timeline_width
            
            # Update line position
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y - 30, 
                x_pos, self.timeline_y + 30
            )
            return True
        except Exception as e:
            if DEBUG_MODE:
                print(f"Error updating progress line: {e}")
            return False
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter."""
        try:
            original_color = self.visual_canvas.itemcget(indicator, "fill")
            self.visual_canvas.itemconfig(indicator, fill=color)
            
            # Schedule color change back
            def reset_color():
                try:
                    if self.visual_canvas.winfo_exists():
                        self.visual_canvas.itemconfig(indicator, fill=original_color)
                except Exception as e:
                    if DEBUG_MODE:
                        print(f"Error resetting indicator color: {e}")
            
            self.root.after(int(duration * 1000), reset_color)
            return True
        except Exception as e:
            if DEBUG_MODE:
                print(f"Error flashing indicator: {e}")
            return False
    
    def flash_click_indicator(self):
        """Flash the click indicator when a click is detected."""
        try:
            if hasattr(self, 'click_indicator') and self.visual_canvas:
                return self.flash_indicator(self.click_indicator, "green")
            return False
        except Exception as e:
            if DEBUG_MODE:
                print(f"Error flashing click indicator: {e}")
            return False
    
    def flash_tactile_indicator(self):
        """Flash the tactile indicator when a tactile stimulus is presented."""
        try:
            if hasattr(self, 'tactile_indicator') and self.visual_canvas:
                return self.flash_indicator(self.tactile_indicator, "yellow")
            return False
        except Exception as e:
            if DEBUG_MODE:
                print(f"Error flashing tactile indicator: {e}")
            return False
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time."""
        try:
            timeline_start_x = 400
            timeline_width = self.timeline_end_x - timeline_start_x
            
            # Calculate max duration based on last trial time
            max_duration = 90  # Default 90 seconds
            if self.trial_timeline:
                max_duration = max(self.trial_timeline.keys()) + 30  # Add 30s buffer
            
            # Calculate position based on time
            x_pos = timeline_start_x + min(time_sec / max_duration, 1.0) * timeline_width
            
            # Create marker
            marker = self.visual_canvas.create_oval(
                x_pos-5, self.timeline_y-5, x_pos+5, self.timeline_y+5, 
                fill=color, outline="black", width=1
            )
            self.timeline_markers.append(marker)
            
            # Flash the timeline area briefly
            flash = self.visual_canvas.create_rectangle(
                x_pos-10, self.timeline_y-10, x_pos+10, self.timeline_y+10, 
                outline="red", width=2
            )
            self.root.after(200, lambda f=flash: self.visual_canvas.delete(f))
            
            return True
        except Exception as e:
            if DEBUG_MODE:
                print(f"Error creating timeline marker: {e}")
            return False
    
    def clear_timeline(self):
        """Clear all markers from the timeline."""
        try:
            for marker in self.timeline_markers:
                self.visual_canvas.delete(marker)
            self.timeline_markers = []
            
            # Add a flash effect to show timeline has been cleared
            timeline_start_x = 400
            flash = self.visual_canvas.create_rectangle(
                timeline_start_x-5, self.timeline_y-20, 
                self.timeline_end_x+5, self.timeline_y+20, 
                outline="blue", width=2
            )
            self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
            
            logging.info("Timeline cleared")
            return True
        except Exception as e:
            logging.error(f"Error clearing timeline: {e}")
            return False
            
    def continuous_progress_update(self, max_duration):
        """Continuously update the progress line during the experiment."""
        try:
            update_interval = 0.05  # Update every 50ms
            
            while not self.stop_flag:
                if self.experiment_start_time is not None and not self.pause_flag:
                    elapsed = time.perf_counter() - self.experiment_start_time
                    if elapsed > max_duration:
                        break
                    # Schedule an update to the progress line
                    self.root.after(0, lambda t=elapsed: self.update_progress_line(t))
                
                # Don't burn CPU
                time.sleep(update_interval)
                
            if DEBUG_MODE:
                print("Progress update thread completed")
                
        except Exception as e:
            logging.error(f"Error in progress update thread: {e}")
    
    def generate_adaptive_audio_segment(self, missed_trials):
        """
        Generate a custom audio segment containing only the missed trials
        that will be inserted before the ending instructions.
        
        Args:
            missed_trials: List of trials where participant didn't respond
            
        Returns:
            Tuple of (looming_audio_segment, tactile_audio_segment, sample_rate)
        """
        try:
            # Get base audio data
            looming_data, looming_sr = sf.read(self.looming_audio_file, dtype='float32')
            tactile_data, tactile_sr = sf.read(self.tactile_audio_file, dtype='float32')
            
            # Create empty arrays for new segments
            # Use a fixed duration per trial (e.g., 5 seconds)
            trial_duration_seconds = 5
            trial_duration_samples = int(trial_duration_seconds * looming_sr)
            total_duration_samples = len(missed_trials) * trial_duration_samples
            
            # Initialize new segments with proper shape
            if len(looming_data.shape) > 1:  # Stereo
                new_looming_segment = np.zeros((total_duration_samples, looming_data.shape[1]), dtype='float32')
            else:  # Mono
                new_looming_segment = np.zeros(total_duration_samples, dtype='float32')
                
            if len(tactile_data.shape) > 1:  # Stereo
                new_tactile_segment = np.zeros((total_duration_samples, tactile_data.shape[1]), dtype='float32')
            else:  # Mono
                new_tactile_segment = np.zeros(total_duration_samples, dtype='float32')
            
            # Load stimulus files
            stimulus_dir = os.path.join(BASE_DIR, "PPS_Experiment_Module")
            looming_stimuli = {}
            for direction in ['right', 'front', 'left']:
                try:
                    path = os.path.join(stimulus_dir, f"{direction}_az0_FABIAN_HRIR_natural.wav")
                    if not os.path.exists(path):
                        # Try alternate naming pattern
                        path = os.path.join(stimulus_dir, f"{direction}_az0_FABIAN_HRIR_natural.wav")
                    
                    looming_stimuli[direction], _ = sf.read(path, dtype='float32')
                    logging.info(f"Loaded {direction} stimulus for adaptive segment")
                except Exception as e:
                    logging.error(f"Error loading {direction} stimulus: {e}")
            
            # Load tactile stimulus
            tactile_stimulus_path = os.path.join(stimulus_dir, "tactile_stimulus.wav")
            try:
                tactile_stimulus, _ = sf.read(tactile_stimulus_path, dtype='float32')
            except Exception as e:
                logging.error(f"Error loading tactile stimulus: {e}")
                tactile_stimulus = None
            
            # Track insertion positions for debugging
            insertion_log = []
            
            # Insert stimuli for each missed trial
            for i, trial in enumerate(missed_trials):
                # Get trial parameters
                trial_type = trial.get('trial_type', 'baseline')
                stimulus_type = trial.get('stimulus_type', 'front')
                soa_ms = trial.get('soa_value_ms', 500)
                
                # Calculate position in the new segments
                start_pos = i * trial_duration_samples
                
                # Add looming stimulus (1 second into trial)
                looming_pos = start_pos + int(1.0 * looming_sr)
                
                if stimulus_type in looming_stimuli:
                    stim_data = looming_stimuli[stimulus_type]
                    
                    # Determine how much of the stimulus can fit
                    copy_length = min(len(stim_data), len(new_looming_segment) - looming_pos)
                    
                    if copy_length > 0:
                        # Handle stereo vs mono
                        if len(new_looming_segment.shape) > 1 and len(stim_data.shape) > 1:
                            # Both stereo
                            new_looming_segment[looming_pos:looming_pos+copy_length] += stim_data[:copy_length]
                        elif len(new_looming_segment.shape) > 1 and len(stim_data.shape) == 1:
                            # Target stereo, source mono
                            for ch in range(new_looming_segment.shape[1]):
                                new_looming_segment[looming_pos:looming_pos+copy_length, ch] += stim_data[:copy_length]
                        elif len(new_looming_segment.shape) == 1 and len(stim_data.shape) > 1:
                            # Target mono, source stereo
                            new_looming_segment[looming_pos:looming_pos+copy_length] += np.mean(stim_data[:copy_length], axis=1)
                        else:
                            # Both mono
                            new_looming_segment[looming_pos:looming_pos+copy_length] += stim_data[:copy_length]
                
                # Add tactile stimulus with SOA if this is not a catch trial
                if trial_type != 'catch' and tactile_stimulus is not None:
                    soa_sec = soa_ms / 1000.0
                    tactile_pos = looming_pos + int(soa_sec * tactile_sr)
                    
                    # Determine how much of the stimulus can fit
                    copy_length = min(len(tactile_stimulus), len(new_tactile_segment) - tactile_pos)
                    
                    if copy_length > 0:
                        # Handle stereo vs mono (same logic as above)
                        if len(new_tactile_segment.shape) > 1 and len(tactile_stimulus.shape) > 1:
                            new_tactile_segment[tactile_pos:tactile_pos+copy_length] += tactile_stimulus[:copy_length]
                        elif len(new_tactile_segment.shape) > 1 and len(tactile_stimulus.shape) == 1:
                            for ch in range(new_tactile_segment.shape[1]):
                                new_tactile_segment[tactile_pos:tactile_pos+copy_length, ch] += tactile_stimulus[:copy_length]
                        elif len(new_tactile_segment.shape) == 1 and len(tactile_stimulus.shape) > 1:
                            new_tactile_segment[tactile_pos:tactile_pos+copy_length] += np.mean(tactile_stimulus[:copy_length], axis=1)
                        else:
                            new_tactile_segment[tactile_pos:tactile_pos+copy_length] += tactile_stimulus[:copy_length]
                
                # Log insertion for debugging
                insertion_log.append({
                    'trial_number': trial.get('trial_number', i),
                    'trial_type': trial_type,
                    'stimulus_type': stimulus_type,
                    'soa_ms': soa_ms,
                    'start_pos_seconds': start_pos / looming_sr,
                    'looming_pos_seconds': looming_pos / looming_sr,
                    'tactile_pos_seconds': tactile_pos / tactile_sr if trial_type != 'catch' else None
                })
                
                logging.info(f"Added missed trial {trial.get('trial_number', i)} to adaptive segment")
            
            # Save insertion log for debugging
            try:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                log_file = os.path.join(
                    DATA_LOG_DIR, 
                    f"{self.participant_initials}_adaptive_insertion_log_{timestamp}.json"
                )
                with open(log_file, 'w') as f:
                    json.dump(insertion_log, f, indent=2)
                logging.info(f"Saved adaptive insertion log to {log_file}")
            except Exception as e:
                logging.error(f"Error saving adaptive insertion log: {e}")
            
            return new_looming_segment, new_tactile_segment, looming_sr
            
        except Exception as e:
            logging.error(f"Error generating adaptive audio segment: {e}")
            return None, None, None
    
    def extract_ending_instructions(self):
        """
        Extract the ending instructions from the original audio file.
        This is the segment after the last trial.
        
        Returns:
            Tuple of (looming_ending, tactile_ending, sample_rate)
        """
        try:
            # Load original audio files
            looming_data, looming_sr = sf.read(self.looming_audio_file, dtype='float32')
            tactile_data, tactile_sr = sf.read(self.tactile_audio_file, dtype='float32')
            
            # For this implementation, we need to know where the last trial ends
            if self.trial_timeline:
                # Get the timestamp of the last trial (maximum key)
                last_time_sec = max(self.trial_timeline.keys())
                
                # Add a buffer (e.g., 10 seconds) to ensure we're past the last trial
                buffer_sec = 10
                end_of_trials_sample = int((last_time_sec + buffer_sec) * looming_sr)
                
                # Create ending segments
                if end_of_trials_sample < len(looming_data):
                    looming_ending = looming_data[end_of_trials_sample:]
                    
                    # Make sure tactile ending has same length
                    if end_of_trials_sample < len(tactile_data):
                        tactile_ending = tactile_data[end_of_trials_sample:]
                    else:
                        # Create silence with same shape as looming_ending
                        if len(looming_ending.shape) > 1:
                            tactile_ending = np.zeros((len(looming_ending), looming_ending.shape[1]), dtype='float32')
                        else:
                            tactile_ending = np.zeros(len(looming_ending), dtype='float32')
                    
                    logging.info(f"Extracted ending instructions: {len(looming_ending)/looming_sr:.2f} seconds")
                    return looming_ending, tactile_ending, looming_sr
            
            # If we can't determine from trial timeline, use a fixed proportion (e.g., last 10%)
            logging.info("Using fixed proportion for ending extraction (10% of total)")
            ending_proportion = 0.10
            end_index = int(len(looming_data) * (1 - ending_proportion))
            
            looming_ending = looming_data[end_index:]
            
            # Make sure tactile ending has same length
            if end_index < len(tactile_data):
                tactile_ending = tactile_data[end_index:]
            else:
                # Create silence with same shape as looming_ending
                if len(looming_ending.shape) > 1:
                    tactile_ending = np.zeros((len(looming_ending), looming_ending.shape[1]), dtype='float32')
                else:
                    tactile_ending = np.zeros(len(looming_ending), dtype='float32')
            
            return looming_ending, tactile_ending, looming_sr
            
        except Exception as e:
            logging.error(f"Error extracting ending instructions: {e}")
            return None, None, None
    
    def create_adaptive_audio_files(self, missed_trials):
        """
        Create adaptive audio files that include missed trials before the ending.
        
        Args:
            missed_trials: List of trials where the participant didn't respond
            
        Returns:
            Tuple of (new_looming_file, new_tactile_file)
        """
        try:
            # Log what we're doing
            logging.info(f"Creating adaptive audio with {len(missed_trials)} missed trials")
            for i, trial in enumerate(missed_trials):
                logging.info(f"  Trial {i+1}: #{trial.get('trial_number', '?')} - {trial.get('trial_type', '?')} - {trial.get('stimulus_type', '?')}")
            
            # Extract ending instructions
            looming_ending, tactile_ending, sample_rate = self.extract_ending_instructions()
            if looming_ending is None:
                raise ValueError("Could not extract ending instructions")
                
            # Generate adaptive segment for missed trials
            adaptive_looming, adaptive_tactile, _ = self.generate_adaptive_audio_segment(missed_trials)
            if adaptive_looming is None:
                raise ValueError("Could not generate adaptive audio segment")
                
            # Load main audio data (without ending)
            looming_data, looming_sr = sf.read(self.looming_audio_file, dtype='float32')
            tactile_data, tactile_sr = sf.read(self.tactile_audio_file, dtype='float32')
            
            # Determine where to cut the main audio (same as extract_ending_instructions)
            if self.trial_timeline:
                last_time_sec = max(self.trial_timeline.keys())
                buffer_sec = 10
                end_of_trials_sample = int((last_time_sec + buffer_sec) * looming_sr)
                
                # Cut the main audio
                if end_of_trials_sample < len(looming_data):
                    main_looming = looming_data[:end_of_trials_sample]
                    main_tactile = tactile_data[:min(end_of_trials_sample, len(tactile_data))]
                else:
                    # Use all of main audio
                    main_looming = looming_data
                    main_tactile = tactile_data
            else:
                # Use fixed proportion (90% of original)
                end_index = int(len(looming_data) * 0.9)
                main_looming = looming_data[:end_index]
                main_tactile = tactile_data[:min(end_index, len(tactile_data))]
                
            # Create transition audio - a brief silence or custom "Now we will repeat some trials" message
            transition_duration_sec = 2.0
            transition_samples = int(transition_duration_sec * sample_rate)
            
            # Create silence with appropriate shape
            if len(main_looming.shape) > 1:  # Stereo
                transition_looming = np.zeros((transition_samples, main_looming.shape[1]), dtype='float32')
            else:  # Mono
                transition_looming = np.zeros(transition_samples, dtype='float32')
                
            if len(main_tactile.shape) > 1:  # Stereo
                transition_tactile = np.zeros((transition_samples, main_tactile.shape[1]), dtype='float32')
            else:  # Mono
                transition_tactile = np.zeros(transition_samples, dtype='float32')
            
            # Combine all segments
            logging.info("Combining audio segments:")
            logging.info(f"  Main: {len(main_looming)/looming_sr:.2f} seconds")
            logging.info(f"  Transition: {transition_duration_sec:.2f} seconds")
            logging.info(f"  Adaptive: {len(adaptive_looming)/looming_sr:.2f} seconds")
            logging.info(f"  Ending: {len(looming_ending)/looming_sr:.2f} seconds")
            
            # Check shapes for compatibility
            if len(main_looming.shape) != len(adaptive_looming.shape):
                logging.warning(f"Shape mismatch: main {main_looming.shape}, adaptive {adaptive_looming.shape}")
                
                # Convert to consistent format (use main_looming's shape as reference)
                if len(main_looming.shape) > 1:  # Main is stereo
                    # Convert adaptive to stereo
                    if len(adaptive_looming.shape) == 1:
                        adaptive_looming = np.column_stack((adaptive_looming, adaptive_looming))
                else:  # Main is mono
                    # Convert adaptive to mono
                    if len(adaptive_looming.shape) > 1:
                        adaptive_looming = np.mean(adaptive_looming, axis=1)
            
            if len(main_tactile.shape) != len(adaptive_tactile.shape):
                logging.warning(f"Shape mismatch: main tactile {main_tactile.shape}, adaptive tactile {adaptive_tactile.shape}")
                
                # Convert to consistent format (use main_tactile's shape as reference)
                if len(main_tactile.shape) > 1:  # Main is stereo
                    # Convert adaptive to stereo
                    if len(adaptive_tactile.shape) == 1:
                        adaptive_tactile = np.column_stack((adaptive_tactile, adaptive_tactile))
                else:  # Main is mono
                    # Convert adaptive to mono
                    if len(adaptive_tactile.shape) > 1:
                        adaptive_tactile = np.mean(adaptive_tactile, axis=1)
            
            # Concatenate segments
            try:
                new_looming = np.concatenate([main_looming, transition_looming, adaptive_looming, looming_ending])
                new_tactile = np.concatenate([main_tactile, transition_tactile, adaptive_tactile, tactile_ending])
            except ValueError as e:
                logging.error(f"Concatenation error: {e}")
                logging.error(f"Shapes: main {main_looming.shape}, trans {transition_looming.shape}, " +
                            f"adaptive {adaptive_looming.shape}, ending {looming_ending.shape}")
                raise
            
            # Create filenames for the new files
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            new_looming_file = os.path.join(
                DATA_LOG_DIR, 
                f"{self.participant_initials}_adaptive_looming_{timestamp}.wav"
            )
            new_tactile_file = os.path.join(
                DATA_LOG_DIR, 
                f"{self.participant_initials}_adaptive_tactile_{timestamp}.wav"
            )
            
            # Save the new files
            sf.write(new_looming_file, new_looming, sample_rate)
            sf.write(new_tactile_file, new_tactile, sample_rate)
            
            logging.info(f"Created adaptive audio files with {len(missed_trials)} repeated trials")
            logging.info(f"New looming file: {new_looming_file}")
            logging.info(f"New tactile file: {new_tactile_file}")
            
            return new_looming_file, new_tactile_file
            
        except Exception as e:
            logging.error(f"Error creating adaptive audio files: {e}")
            traceback.print_exc()
            return None, None
    
    def get_missed_trials(self):
        """
        Analyze responses and identify missed trials.
        
        Returns:
            List of dictionaries with missed trial information
        """
        missed_trials = []
        
        try:
            # We need to track responses to tactile stimuli
            response_windows = {}  # Maps trial numbers to response windows
            responses_by_trial = {}  # Maps trial numbers to actual responses
            
            # First, build timeline of tactile stimuli and their response windows
            for time_sec, trial_info in sorted(self.trial_timeline.items()):
                # Skip catch trials
                if trial_info.get('trial_type') == 'catch':
                    continue
                    
                # Only process tactile stimulus points
                if trial_info.get('is_tactile', False):
                    trial_number = trial_info.get('trial_number')
                    
                    # Define response window for this trial
                    response_windows[trial_number] = {
                        'start': time_sec,
                        'end': time_sec + RESPONSE_WINDOW,
                        'info': trial_info
                    }
            
            # Group responses by trial
            for response in self.responses:
                response_time = response.get('time')
                
                # Find which trial this response belongs to
                for trial_number, window in response_windows.items():
                    if window['start'] <= response_time <= window['end']:
                        if trial_number not in responses_by_trial:
                            responses_by_trial[trial_number] = []
                        
                        responses_by_trial[trial_number].append(response)
                        break
            
            # Find trials with no responses
            for trial_number, window in response_windows.items():
                if trial_number not in responses_by_trial or not responses_by_trial[trial_number]:
                    # This is a missed trial
                    missed_trials.append({
                        'trial_number': trial_number,
                        'trial_type': window['info'].get('trial_type', 'unknown'),
                        'stimulus_type': window['info'].get('stimulus_type', 'unknown'),
                        'soa_value_ms': window['info'].get('soa_value_ms', 0)
                    })
            
            # Randomly shuffle the missed trials to vary the order
            random.shuffle(missed_trials)
            
            logging.info(f"Found {len(missed_trials)} missed trials out of {len(response_windows)} tactile trials")
            
        except Exception as e:
            logging.error(f"Error analyzing missed trials: {e}")
            traceback.print_exc()
        
        return missed_trials
    
    def run_experiment_with_adaptive_trials(self):
        """
        Run the full experiment with adaptive trials for missed responses.
        
        This method:
        1. Plays the main experiment
        2. Analyzes missed trials
        3. Generates adaptive audio with missed trials 
        4. Plays the adaptive segment
        
        Returns:
            Boolean indicating success
        """
        try:
            # Create response CSV file
            self.create_response_csv()
            
            # Reset flags and counters
            self.stop_flag = False
            self.pause_flag = False
            self.responses = []
            self.missed_trials = []
            self.click_count = 0
            self.clear_timeline()
            
            # Set the experiment start time
            self.experiment_start_time = time.perf_counter()
            
            # Send start markers to LSL
            if self.lsl_enabled and self.marker_outlet:
                self.marker_outlet.push_sample(["0.000", "experiment_start", self.participant_initials])
                logging.info("Sent LSL marker: experiment_start")
            
            # Play main experiment
            logging.info("Starting main experiment audio playback")
            main_completed = self.play_audio_files(is_adaptive=False)
            
            if not main_completed or self.stop_flag:
                logging.info("Main experiment did not complete normally, skipping adaptive trials")
                return False
            
            # Analyze responses and find missed trials
            self.missed_trials = self.get_missed_trials()
            
            # If we have missed trials, play the adaptive segment
            if self.missed_trials:
                # Create adaptive audio files
                adaptive_looming, adaptive_tactile = self.create_adaptive_audio_files(self.missed_trials)
                
                if adaptive_looming and adaptive_tactile:
                    # Show a message about repeating missed trials
                    def show_repeat_message():
                        messagebox.showinfo(
                            "Repeating Missed Trials", 
                            f"Now repeating {len(self.missed_trials)} missed trials.\n\nPlease continue responding as before."
                        )
                    
                    # Run in main thread to avoid blocking
                    if hasattr(self, 'root') and self.root.winfo_exists():
                        self.root.after(0, show_repeat_message)
                    
                    # Wait for the message to be dismissed
                    time.sleep(0.5)
                    
                    # Store original files
                    orig_looming = self.looming_audio_file
                    orig_tactile = self.tactile_audio_file
                    
                    # Set to adaptive files
                    self.looming_audio_file = adaptive_looming
                    self.tactile_audio_file = adaptive_tactile
                    
                    # Play adaptive segment
                    logging.info("Starting adaptive segment audio playback")
                    adaptive_completed = self.play_audio_files(is_adaptive=True)
                    
                    # Restore original files
                    self.looming_audio_file = orig_looming
                    self.tactile_audio_file = orig_tactile
                    
                    return adaptive_completed
                else:
                    logging.error("Failed to create adaptive audio files")
                    return False
            else:
                logging.info("No missed trials to repeat")
                return True
            
        except Exception as e:
            logging.error(f"Error running experiment with adaptive trials: {e}")
            traceback.print_exc()
            return False
    
    def play_audio_files(self, is_adaptive=False):
        """
        Play both audio files simultaneously, supporting pause/resume.
        
        Args:
            is_adaptive: Boolean indicating if this is the adaptive segment
            
        Returns:
            Boolean indicating success
        """
        try:
            # Load audio files
            looming_data, looming_sr = sf.read(self.looming_audio_file, dtype='float32')
            tactile_data, tactile_sr = sf.read(self.tactile_audio_file, dtype='float32')
            
            logging.info(f"Loaded {'adaptive' if is_adaptive else 'main'} audio files:")
            logging.info(f"  Looming: {os.path.basename(self.looming_audio_file)}")
            logging.info(f"  Sample rate: {looming_sr} Hz, Duration: {len(looming_data)/looming_sr:.2f} seconds")
            
            logging.info(f"  Tactile: {os.path.basename(self.tactile_audio_file)}")
            logging.info(f"  Sample rate: {tactile_sr} Hz, Duration: {len(tactile_data)/tactile_sr:.2f} seconds")
            
            # Ensure we have matching sample rates
            if looming_sr != tactile_sr:
                logging.warning(f"Sample rates don't match! Looming: {looming_sr}Hz, Tactile: {tactile_sr}Hz")
                logging.warning("This might cause synchronization issues")
            
            # Store the data for pause/resume
            self.looming_data = looming_data
            self.looming_sr = looming_sr
            self.tactile_data = tactile_data
            self.tactile_sr = tactile_sr
            
            # Set up callback functions for streaming
            looming_pos = 0
            tactile_pos = 0
            
            # Looming callback
            def looming_callback(outdata, frames, time, status):
                nonlocal looming_pos
                if status:
                    logging.warning(f"Looming audio status: {status}")
                
                # Check if paused
                if self.pause_flag:
                    # Fill with silence
                    outdata.fill(0)
                    return
                
                if self.stop_flag:
                    raise sd.CallbackStop
                
                # Check if we've reached the end
                if looming_pos >= len(self.looming_data):
                    raise sd.CallbackStop
                
                # Copy data to output
                if len(self.looming_data.shape) == 1:  # Mono
                    remain = min(frames, len(self.looming_data) - looming_pos)
                    outdata[:remain, 0] = self.looming_data[looming_pos:looming_pos+remain]
                    if remain < frames:
                        outdata[remain:, 0] = 0
                else:  # Stereo
                    remain = min(frames, len(self.looming_data) - looming_pos)
                    outdata[:remain] = self.looming_data[looming_pos:looming_pos+remain]
                    if remain < frames:
                        outdata[remain:] = 0
                
                looming_pos += frames
            
            # Tactile callback
            def tactile_callback(outdata, frames, time, status):
                nonlocal tactile_pos
                if status:
                    logging.warning(f"Tactile audio status: {status}")
                
                # Check if paused
                if self.pause_flag:
                    # Fill with silence
                    outdata.fill(0)
                    return
                
                if self.stop_flag:
                    raise sd.CallbackStop
                
                # Check if we've reached the end
                if tactile_pos >= len(self.tactile_data):
                    raise sd.CallbackStop
                
                # Copy data to output
                if len(self.tactile_data.shape) == 1:  # Mono
                    remain = min(frames, len(self.tactile_data) - tactile_pos)
                    outdata[:remain, 0] = self.tactile_data[tactile_pos:tactile_pos+remain]
                    if remain < frames:
                        outdata[remain:, 0] = 0
                else:  # Stereo
                    remain = min(frames, len(self.tactile_data) - tactile_pos)
                    outdata[:remain] = self.tactile_data[tactile_pos:tactile_pos+remain]
                    if remain < frames:
                        outdata[remain:] = 0
                
                tactile_pos += frames
                
                # Check for tactile stimuli to animate the timeline
                # This is approximate - we're checking if we just played frames that
                # might contain a tactile stimulus
                if not is_adaptive and self.experiment_start_time is not None:  # Only do this for main experiment
                    current_time = time.perf_counter() - self.experiment_start_time
                    
                    if current_time:
                        # Find any trial with a tactile stimulus in this frame range
                        frame_start_time = current_time - (frames / tactile_sr)
                        frame_end_time = current_time
                        
                        for time_sec, trial_info in self.trial_timeline.items():
                            if frame_start_time <= time_sec <= frame_end_time and trial_info.get('is_tactile', False):
                                # Flash tactile indicator on UI
                                self.root.after(0, self.flash_tactile_indicator)
                                
                                # Add marker to timeline
                                self.root.after(0, lambda t=time_sec: self.add_timeline_marker(t, "blue"))
                                
                                # Send marker to LSL
                                if self.lsl_enabled and self.marker_outlet:
                                    self.marker_outlet.push_sample([
                                        f"{time_sec:.3f}",
                                        f"tactile_stimulus_{trial_info.get('trial_number', 'unknown')}",
                                        self.participant_initials
                                    ])
                                
                                break